# Aim

Fit an autoregressive model to Keck TT data, and see how good of a description it is.

In [3]:
using DSP, Distributions, LinearAlgebra
using Plots, NPZ
import Statistics: mean
rms(x) = x .^ 2 |> mean |> sqrt

rms (generic function with 1 method)

In [2]:
include("../src/kfilter.jl")

simulate (generic function with 1 method)

In [ ]:
σ = 0.06
pols = npzread("../data/sims/atm_openloops.npy")[:,1]
measurements = pols .+ rand(Normal(0, σ), (length(pols),))
plot(pols, xlabel="Time", ylabel="Tip (mas)")

In [ ]:
rms(pols .- mean(pols))

In [ ]:
ar_len = 2
n = length(pols)

TTs_mat = Matrix{Float64}(undef, n - ar_len, ar_len)
for i = 1:ar_len
   TTs_mat[:, i] = pols[ar_len - i + 1 : n - i, 1] 
end

# Solve autoregressive problem
ar_coef = TTs_mat \ pols[ar_len + 1: end]

In [ ]:
ar_residual = pols[ar_len:end-1] .- (TTs_mat * evaluate(ar_coef));

In [ ]:
nsteps_plot = 500
plot(pols[ar_len:nsteps_plot+ar_len-1], label="HCIPy open-loops")
plot!((TTs_mat * ar_coef)[1:nsteps_plot], label="AR fit"; 
    title="autoregression: RMS error = $(round(rms(ar_residual); digits=3))")
plot!(ar_residual[1:nsteps_plot], label="Residual")

In [ ]:
plot_per!(p::DSP.Periodograms.Periodogram; kwargs...) = plot!(p.freq[1000:end], p.power[1000:end], yaxis=:log; kwargs...)

In [ ]:
plot()
periodogram(ar_residual, fs=1000) |> plot_per!

In [ ]:
A = zeros(ar_len, ar_len)
A[1,:] = ar_coef
for i in 2:ar_len
    A[i,i-1] += 1.0
end

B = zeros(ar_len, 0)
C = zeros(1, ar_len)
C[1] += 1

Q = zeros(ar_len, ar_len)
Q[1,1] = mean(ar_residual[1:100] .^ 2)

R = σ^2 * I

kf = KFilter(A, B, C, Q, R)

In [ ]:
x = [pols[99], pols[100]]
simulate(kf, pols[1001:end], [[] for _ in pols[1001:end]]) 